# FITS Playground
---

In [1]:
from astropy.io import fits
import fits_pb2 as fp

In [2]:
fits_image_filename = fits.util.get_testdata_filepath('test0.fits')

In [3]:
hdul = fits.open(fits_image_filename)

In [4]:
hdul.info()

Filename: C:\Users\ntoner\AppData\Local\Continuum\anaconda3\lib\site-packages\astropy\io\fits\tests\data\test0.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     138   ()      
  1  SCI           1 ImageHDU        61   (40, 40)   int16   
  2  SCI           2 ImageHDU        61   (40, 40)   int16   
  3  SCI           3 ImageHDU        61   (40, 40)   int16   
  4  SCI           4 ImageHDU        61   (40, 40)   int16   


In [5]:
hdu_header = hdul[0].header

In [6]:
hdu_header

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                   16 / number of bits per data pixel                  
NAXIS   =                    0 / number of data axes                            
EXTEND  =                    T / FITS dataset may contain extensions            
GROUPS  =                    F / data has groups                                
NEXTEND =                    4 / Number of standard extensions                  
BSCALE  =           1.000000E0 / REAL = TAPE*BSCALE + BZERO                     
BZERO   =           3.276800E4 /                                                
ORIGIN  = 'NOAO-IRAF FITS Image Kernel Aug 1 1997' / FITS file originator       
DATE    = '01/04/99  '         / Date FITS file was generated                   
IRAF-TLM= 'xxx     '              / Time of last modification                   
                                                                                
              / GROUP PARAME

In [7]:
def make_header_list(hdu_header):
    header_list = []
    for k, v in zip(hdu_header.keys(), hdu_header.values()):
        if k is '':
            header_list[-1]['comment'] += f'\n{v.strip().strip("/")}'
        else:
            header_list.append({k: v, 'comment': hdu_header.comments[k]})
    return header_list

In [8]:
header_list = make_header_list(hdul[0].header)
header_list

[{'SIMPLE': True, 'comment': 'file does conform to FITS standard'},
 {'BITPIX': 16, 'comment': 'number of bits per data pixel'},
 {'NAXIS': 0, 'comment': 'number of data axes'},
 {'EXTEND': True, 'comment': 'FITS dataset may contain extensions'},
 {'GROUPS': False, 'comment': 'data has groups'},
 {'NEXTEND': 4, 'comment': 'Number of standard extensions'},
 {'BSCALE': 1.0, 'comment': 'REAL = TAPE*BSCALE + BZERO'},
 {'BZERO': 32768.0, 'comment': ''},
 {'ORIGIN': 'NOAO-IRAF FITS Image Kernel Aug 1 1997',
  'comment': 'FITS file originator'},
 {'DATE': '01/04/99', 'comment': 'Date FITS file was generated'},
 {'IRAF-TLM': 'xxx',
  'comment': 'Time of last modification\n\n GROUP PARAMETERS: OSS\n\n GROUP PARAMETERS: PODPS\n\n GROUP PARAMETERS: DATA QUALITY FILE SUMMARY\n\n GROUP PARAMETERS: PHOTOMETRY\n\n GROUP PARAMETERS: IMAGE STATISTICS\n\n WFPC-II DATA DESCRIPTOR KEYWORDS'},
 {'INSTRUME': 'WFPC2',
  'comment': 'identifier for instrument used to acquire data'},
 {'ROOTNAME': 'U2EQ0201T', 

In [9]:
fits_fp = fp.Fits()

In [10]:
hdu_fp = fp.Hdu()

In [11]:
header_fp = fp.Header()

In [12]:
list(header_list[0].keys())[0]

'SIMPLE'

In [13]:
list(header_list[0].values())[1]

'file does conform to FITS standard'

In [14]:
header_list[0][list(header_list[0].keys())[0]]

True

In [15]:
def build_pair(pair, keys, values):
    pair.key = keys[0]
    pair.value = values[0]
    pair.comment = values[1]
    return pair

In [16]:
import sys

In [17]:
sys.float_info.max

1.7976931348623157e+308

In [18]:
float32_max = (2 - 2**-23) * 2**127
float32_max

3.4028234663852886e+38

In [19]:
dict_fp = fp.Dictionary()

In [20]:
def parse_fits_header(header_fp, header_list):
    header_fp.Clear()
    for d in header_list:
        keys = list(d.keys())
        values = list(d.values())
        if keys[0].lower() == 'simple':
            header_fp.simple = values[0]
        elif keys[0].lower() == 'bitpix':
            header_fp.bitpix = values[0]
        elif 'naxis' in keys[0].lower():
            header_fp.size.append(values[0])
        elif keys[0].lower() == 'comment':
            header_fp.comment += f'\n{values[0].strip()}'
        elif keys[0].lower() == 'history':
            header_fp.history += f'\n{values[0].strip()}'
        elif keys[0].lower() == 'xtension':
            header_fp.extension = values[0]
        elif keys[0].lower() == 'nextend':
            header_fp.n_extensions = values[0]
        else:
            if isinstance(values[0], str):
                string_pair = dict_fp.StringPair()
                string_pair = build_pair(string_pair, keys, values)
                header_fp.keywords.string_pairs.extend([string_pair])
            elif isinstance(values[0], bool):
                bool_pair = dict_fp.BoolPair()
                bool_pair = build_pair(bool_pair, keys, values)
                header_fp.keywords.bool_pairs.extend([bool_pair])
            elif isinstance(values[0], int):
                if values[0] < 0:
                    if values[0] < -2**31:
                        int_pair = dict_fp.SInt64Pair()
                        int_pair = build_pair(int_pair, keys, values)
                        header_fp.keywords.sint64_pairs.extend([int_pair])
                    else:
                        int_pair = dict_fp.SInt32Pair()
                        int_pair = build_pair(int_pair, keys, values)
                        header_fp.keywords.sint32_pairs.extend([int_pair])
                else:
                    if values[0] > 2**32:
                        int_pair = dict_fp.Int64Pair()
                        int_pair = build_pair(int_pair, keys, values)
                        header_fp.keywords.int64_pairs.extend([int_pair])
                    else:
                        int_pair = dict_fp.Int32Pair()
                        int_pair = build_pair(int_pair, keys, values)
                        header_fp.keywords.int32_pairs.extend([int_pair])
            elif isinstance(values[0], float):
                if values[0] < -float32_max or values[0] > float32_max:
                    # double case
                    float_pair = dict_fp.DoublePair()
                    float_pair = build_pair(float_pair, keys, values)
                    header_fp.keywords.double_pairs.extend([float_pair])
                else:
                    # float case
                    float_pair = dict_fp.FloatPair()
                    float_pair = build_pair(float_pair, keys, values)
                    header_fp.keywords.float_pairs.extend([float_pair])

In [21]:
parse_fits_header(header_fp, header_list)

In [22]:
hdu_fp.header.extend([header_fp])

In [23]:
fits_fp.hdu.extend([hdu_fp])

In [24]:
header_fp.Clear()
hdu_fp.Clear()

In [25]:
parse_fits_header(header_fp, make_header_list(hdul[1].header))

In [26]:
header_fp

bitpix: 16
size: 2
size: 40
size: 40
extension: "IMAGE"
keywords {
  string_pairs {
    key: "EXTNAME"
    value: "SCI"
    comment: "extension name"
  }
  string_pairs {
    key: "ROOTNAME"
    value: "U2EQ0201T"
    comment: "rootname of the observation set"
  }
  string_pairs {
    key: "EXPNAME"
    value: "U2EQ0201T"
    comment: "9 character exposure identifier"
  }
  string_pairs {
    key: "CTYPE1"
    value: "UNITLESS"
    comment: "first coordinate type"
  }
  string_pairs {
    key: "CTYPE2"
    value: "LINE"
    comment: "second coordinate type"
  }
  string_pairs {
    key: "PHOTMODE"
    comment: "Photometry mode"
  }
  int32_pairs {
    key: "PCOUNT"
    comment: "required keyword; must = 0"
  }
  int32_pairs {
    key: "GCOUNT"
    value: 1
    comment: "required keyword; must = 1"
  }
  int32_pairs {
    key: "EXTVER"
    value: 1
    comment: "extension version number"
  }
  int32_pairs {
    key: "FILLCNT"
    comment: "number of segments containing fill"
  }
  int32

In [27]:
hdu_fp.header.extend([header_fp])

In [28]:
hdul[1].data

array([[313, 312, 313, ..., 312, 313, 313],
       [315, 315, 313, ..., 312, 314, 313],
       [313, 313, 312, ..., 314, 312, 313],
       ...,
       [314, 313, 312, ..., 312, 313, 311],
       [313, 315, 312, ..., 313, 313, 312],
       [312, 314, 314, ..., 314, 311, 314]], dtype=int16)

In [29]:
hdu_fp.int32_data.extend(hdul[1].data.ravel())

In [31]:
hdu_fp

header {
  bitpix: 16
  size: 2
  size: 40
  size: 40
  extension: "IMAGE"
  keywords {
    string_pairs {
      key: "EXTNAME"
      value: "SCI"
      comment: "extension name"
    }
    string_pairs {
      key: "ROOTNAME"
      value: "U2EQ0201T"
      comment: "rootname of the observation set"
    }
    string_pairs {
      key: "EXPNAME"
      value: "U2EQ0201T"
      comment: "9 character exposure identifier"
    }
    string_pairs {
      key: "CTYPE1"
      value: "UNITLESS"
      comment: "first coordinate type"
    }
    string_pairs {
      key: "CTYPE2"
      value: "LINE"
      comment: "second coordinate type"
    }
    string_pairs {
      key: "PHOTMODE"
      comment: "Photometry mode"
    }
    int32_pairs {
      key: "PCOUNT"
      comment: "required keyword; must = 0"
    }
    int32_pairs {
      key: "GCOUNT"
      value: 1
      comment: "required keyword; must = 1"
    }
    int32_pairs {
      key: "EXTVER"
      value: 1
      comment: "extension version num

In [32]:
fits_fp.hdu.extend([hdu_fp])

In [33]:
fits_fp

hdu {
  header {
    simple: true
    bitpix: 16
    size: 0
    n_extensions: 4
    keywords {
      string_pairs {
        key: "ORIGIN"
        value: "NOAO-IRAF FITS Image Kernel Aug 1 1997"
        comment: "FITS file originator"
      }
      string_pairs {
        key: "DATE"
        value: "01/04/99"
        comment: "Date FITS file was generated"
      }
      string_pairs {
        key: "IRAF-TLM"
        value: "xxx"
        comment: "Time of last modification\n\n GROUP PARAMETERS: OSS\n\n GROUP PARAMETERS: PODPS\n\n GROUP PARAMETERS: DATA QUALITY FILE SUMMARY\n\n GROUP PARAMETERS: PHOTOMETRY\n\n GROUP PARAMETERS: IMAGE STATISTICS\n\n WFPC-II DATA DESCRIPTOR KEYWORDS"
      }
      string_pairs {
        key: "INSTRUME"
        value: "WFPC2"
        comment: "identifier for instrument used to acquire data"
      }
      string_pairs {
        key: "ROOTNAME"
        value: "U2EQ0201T"
        comment: "rootname of the observation set"
      }
      string_pairs {
        ke

In [34]:
header_fp.Clear()
hdu_fp.Clear()